In [384]:
import os
import sys
import re
import pandas as pd
import numpy as np
from html import unescape
#using email module and policy function (in email) in python to parse mails
import email
import email.policy
input_dir = 'data'


# Enron email dataset

## Loading raw data

In [182]:
dfe = pd.read_csv(os.path.join(input_dir,'enron','emails.csv'))

In [183]:
print(dfe.shape)
dfe.head()

(517401, 2)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [184]:
# A single message looks like this
print(dfe['message'][10])

Message-ID: <33076797.1075855687515.JavaMail.evans@thyme>
Date: Mon, 16 Oct 2000 06:42:00 -0700 (PDT)
From: phillip.allen@enron.com
To: buck.buckner@honeywell.com
Subject: Re: FW: fixed forward or other Collar floor gas price terms
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: "Buckner, Buck" <buck.buckner@honeywell.com> @ ENRON
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail
X-Origin: Allen-P
X-FileName: pallen.nsf

Mr. Buckner,

 For delivered gas behind San Diego, Enron Energy Services is the appropriate 
Enron entity.  I have forwarded your request to Zarin Imam at EES.  Her phone 
number is 713-853-7107.  

Phillip Allen


In [185]:
# get shape of the data
dfe.shape

(517401, 2)

In [186]:
## Helper functions

# concatenate multiple body sessions
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        ctype = part.get_content_type()
        if ctype == 'text/plain' :
            parts.append( part.get_payload() )
        elif ctype == 'text/html' :
            parts.append( html_to_plain_text(part.get_payload() ))
        
    return ''.join(parts)


# convert HTML to plain text
def html_to_plain_text(html):
    text = re.sub('<head.*?>.*?</head>', '', html, flags=re.M | re.S | re.I)
    text = re.sub('<a\s.*?>', ' HYPERLINK ', text, flags=re.M | re.S | re.I)
    text = re.sub('<.*?>', '', text, flags=re.M | re.S)
    text = re.sub(r'(\s*\n)+', '\n', text, flags=re.M | re.S)
    return unescape(text)


In [187]:
# transform the email into correct format
message = dfe['message'][10]
e = email.message_from_string(message)

e.items()

[('Message-ID', '<33076797.1075855687515.JavaMail.evans@thyme>'),
 ('Date', 'Mon, 16 Oct 2000 06:42:00 -0700 (PDT)'),
 ('From', 'phillip.allen@enron.com'),
 ('To', 'buck.buckner@honeywell.com'),
 ('Subject', 'Re: FW: fixed forward or other Collar floor gas price terms'),
 ('Mime-Version', '1.0'),
 ('Content-Type', 'text/plain; charset=us-ascii'),
 ('Content-Transfer-Encoding', '7bit'),
 ('X-From', 'Phillip K Allen'),
 ('X-To', '"Buckner, Buck" <buck.buckner@honeywell.com> @ ENRON'),
 ('X-cc', ''),
 ('X-bcc', ''),
 ('X-Folder', "\\Phillip_Allen_Dec2000\\Notes Folders\\'sent mail"),
 ('X-Origin', 'Allen-P'),
 ('X-FileName', 'pallen.nsf')]

In [188]:
# show message body
e.get_payload()

'Mr. Buckner,\n\n For delivered gas behind San Diego, Enron Energy Services is the appropriate \nEnron entity.  I have forwarded your request to Zarin Imam at EES.  Her phone \nnumber is 713-853-7107.  \n\nPhillip Allen'

## Extract fields from email message object 

In [189]:
# now we add those fields into our 'df' dataframe
# @TODO refactor to run once for an array of fields
def get_field(field, messages):
    column = []
    for message in messages:
        e = email.message_from_string(message)
        column.append(e.get(field))
    return column

In [190]:
dfe.loc[:,('to')] = get_field("To", dfe['message'])
dfe.loc[:,('from')] = get_field("From", dfe['message'])
dfe.loc[:,('date')] = get_field("Date", dfe['message'])
dfe.loc[:,('subject')] = get_field("Subject", dfe['message'])
dfe.head(3)

,file,message,to,from,date,subject
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,tim.belden@enron.com,phillip.allen@enron.com,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,john.lavorato@enron.com,phillip.allen@enron.com,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Re:
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,leah.arsdall@enron.com,phillip.allen@enron.com,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",Re: test


In [191]:
# Extract Message Body
# @TODO refactor to run once for an array of fields
def get_body(messages):
    column = []
    for message in messages:
        e = email.message_from_string(message)
        column.append(get_text_from_email(e))
    return column


In [192]:
dfe.describe()

,file,message,to,from,date,subject
count,517401,517401,495554,517401,517401,517401
unique,517401,517401,58563,20328,224128,159290
top,lavorato-j/deleted_items/492.,Message-ID: <3975082.1075845627684.JavaMail.ev...,pete.davis@enron.com,kay.mann@enron.com,"Wed, 27 Jun 2001 16:02:00 -0700 (PDT)",
freq,1,1,9155,16735,1118,19187


In [193]:
dfe['body'] = get_body(dfe['message'])
dfe.head(3)

,file,message,to,from,date,subject,body
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,tim.belden@enron.com,phillip.allen@enron.com,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",,Here is our forecast\n\n
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,john.lavorato@enron.com,phillip.allen@enron.com,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Re:,Traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,leah.arsdall@enron.com,phillip.allen@enron.com,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",Re: test,test successful. way to go!!!


In [194]:
dfe['body'].head(15)


0                             Here is our forecast\n\n 
1     Traveling to have a business meeting takes the...
2                        test successful.  way to go!!!
3     Randy,\n\n Can you send me a schedule of the s...
4                   Let's shoot for Tuesday at 11:45.  
5     Greg,\n\n How about either next Tuesday or Thu...
6     Please cc the following distribution list with...
7                      any morning between 10 and 11:30
8     1. login:  pallen pw: ke9davis\n\n I don't thi...
9     ---------------------- Forwarded by Phillip K ...
10    Mr. Buckner,\n\n For delivered gas behind San ...
11    Lucy,\n\n Here are the rentrolls:\n\n\n\n Open...
12    ---------------------- Forwarded by Phillip K ...
13    ---------------------- Forwarded by Phillip K ...
14    Dave, \n\n Here are the names of the west desk...
Name: body, dtype: object

## Cleaning and Preparations

In [ ]:
# convert field to date format
dfe['date'] = pd.to_datetime(dfe['date'], utc=True, infer_datetime_format=True)

In [331]:
dfe.dtypes

file                    object
message                 object
to                      object
from                    object
date       datetime64[ns, UTC]
subject                 object
body                    object
dtype: object

In [325]:
dfe.describe()

,file,message,to,from,date,subject,body
count,517401,517401,495554,517401,517401,517401,517401
unique,517401,517401,58563,20328,224122,159290,249025
top,lavorato-j/deleted_items/492.,Message-ID: <3975082.1075845627684.JavaMail.ev...,pete.davis@enron.com,kay.mann@enron.com,2001-06-27 23:02:00+00:00,,"As you know, Enron Net Works (ENW) and Enron G..."
freq,1,1,9155,16735,1118,19187,112
first,NaN,NaN,NaN,NaN,1980-01-01 00:00:00+00:00,NaN,NaN
last,NaN,NaN,NaN,NaN,2044-01-04 22:48:58+00:00,NaN,NaN


In [324]:
# check for null values
print(sum(dfe.body == ''))
print(sum(pd.isna(dfe.date)))
print(sum(dfe.to == ''))
print(sum(dfe['from'] == ''))

0
0
0
0


## Save new dataset with extracted fields

In [319]:
dfe.loc[:, ('date', 'to', 'from', 'body')].to_csv (os.path.join(input_dir, 'enron_clean.csv'), index = False, header=True)

# SpamAssassin dataset

In [155]:
email_path = os.path.join(input_dir, 'spamassassin', 'all_email')
email_path

'data\\spamassassin\\all_email'

In [156]:
len(os.listdir(email_path))

4197

In [157]:
# helper code to investigate to find index of files
# d = pd.Series(sorted(os.listdir(email_path)))
# d.head(50)

In [158]:
email_filenames = [name for name in sorted(os.listdir(email_path)) if len(name) > 20]
len(email_filenames)

4197

In [159]:
def load_email(filename, path):
    with open(os.path.join(path, filename), "rb") as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

emails = [load_email(name, email_path) for name in email_filenames]

In [160]:
len(emails)

4197

In [161]:
print(emails[0])

Return-Path: <ilug-admin@linux.ie>
Delivered-To: yyyy@localhost.netnoteinc.com
Received: from localhost (localhost [127.0.0.1])
	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id 9E1F5441DD
	for <jm@localhost>; Tue,  6 Aug 2002 06:48:09 -0400 (EDT)
Received: from phobos [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for jm@localhost (single-drop); Tue, 06 Aug 2002 11:48:09 +0100 (IST)
Received: from lugh.tuatha.org (root@lugh.tuatha.org [194.125.145.45]) by
    dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g72LqWv13294 for
    <jm-ilug@jmason.org>; Fri, 2 Aug 2002 22:52:32 +0100
Received: from lugh (root@localhost [127.0.0.1]) by lugh.tuatha.org
    (8.9.3/8.9.3) with ESMTP id WAA31224; Fri, 2 Aug 2002 22:50:17 +0100
Received: from bettyjagessar.com (w142.z064000057.nyc-ny.dsl.cnc.net
    [64.0.57.142]) by lugh.tuatha.org (8.9.3/8.9.3) with ESMTP id WAA31201 for
    <ilug@linux.ie>; Fri, 2 Aug 2002 22:50:11 +0100
X-Authentication-Warning: lugh.tuatha.org: Host
 w142.z06

In [162]:
dfa = pd.DataFrame()
dfa['to'] = [message.get('To') for message in emails ]
dfa['from'] = [message.get('From') for message in emails ]
dfa['date'] = [message.get('Date') for message in emails ]
dfa['subject'] = [message.get('Subject') for message in emails ]
dfa['body'] = [get_text_from_email(message) for message in emails ]


In [163]:
dfa.describe()

,to,from,date,subject,body
count,4028,4197,4197,4196,4197
unique,1134,1882,3786,3061,3878
top,yyyy@example.com,boingboing <rssfeeds@example.com>,"Tue, 08 Oct 2002 08:01:06 -0000",[Spambayes] test sets?,\n \n Dear\n Hom...
freq,638,114,6,27,8


In [326]:
dfa.head()

,to,from,date,subject,body
0,ilug@linux.ie,Start Now <startnow2002@hotmail.com>,"Fri, 02 Aug 2002 23:37:59 +0530",[ILUG] STOP THE MLM INSANITY,Greetings!\n\nYou are receiving this letter be...
1,ranmoore@cybertime.net,lmrn@mailexcite.com,"Mon, 28 Jul 1980 14:01:35 -0000","Real Protection, Stun Guns! Free Shipping! Ti...","\nThe Need For Safety Is Real In 2002, You Mig..."
2,cbmark@cbmark.com,amknight@mailexcite.com,"Wed, 30 Jul 1980 18:25:49 -0000","New Improved Fat Burners, Now With TV Fat Abso...","\n*****Bonus Fat Absorbers As Seen On TV, Incl..."
3,ranmoore@swbell.net,jordan23@mailexcite.com,"Thu, 31 Jul 1980 07:20:54 -0000","New Improved Fat Burners, Now With TV Fat Abso...","\n*****Bonus Fat Absorbers As Seen On TV, Incl..."
4,yyyy@pluriproj.pt,yyyy@pluriproj.pt,"Sun, 19 Oct 1980 10:55:16 -0000","Never Repay Cash Grants, $500 - $50,000, Secre...","\nGovernment Grants E-Book 2002\nedition, Just..."


## Cleaning and Preparation

In [365]:
# replace invalid year information
dfa['date'] = dfa['date'].str.replace(" 0102 ", " 2010 ")

In [368]:
# convert field to date format
dfa['date'] = pd.to_datetime(dfa['date'], utc=True, infer_datetime_format=True)

In [376]:
dfa.dtypes

to                      object
from                    object
date       datetime64[ns, UTC]
subject                 object
body                    object
dtype: object

In [377]:
# remove rows without body
dfa = dfa[dfa.body != '']

In [378]:
dfa.describe()

,to,from,date,subject,body
count,4026,4195,4195,4194,4195
unique,1134,1880,3783,3059,3877
top,yyyy@example.com,boingboing <rssfeeds@example.com>,2002-10-08 08:01:22+00:00,[Spambayes] test sets?,\n \n Dear\n Hom...
freq,638,114,6,27,8
first,NaN,NaN,1980-07-28 14:01:35+00:00,NaN,NaN
last,NaN,NaN,2028-10-04 16:05:01+00:00,NaN,NaN


In [382]:
print(sum(dfa.body == ''))
print(sum(pd.isna(dfa.date )))
print(sum(dfa.to == ''))
print(sum(dfa['from'] == ''))

0
0
30
1


## Save new dataset with extracted fields

In [383]:
dfa.loc[:, ('date', 'to', 'from', 'body')].to_csv (os.path.join(input_dir, 'spamassassin_clean.csv'), index = False, header=True)